In [44]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import scorecardpy as sc
import pprint

from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [45]:
df_main = pd.read_csv('filtered_data_iter2.csv')
df = df_main.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114755 entries, 0 to 114754
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                114755 non-null  int64  
 1   STATUS                    114755 non-null  int64  
 2   NAME_CONTRACT_TYPE        114755 non-null  object 
 3   FLAG_OWN_CAR              114755 non-null  object 
 4   FLAG_OWN_REALTY           114755 non-null  object 
 5   AMT_INCOME_TOTAL          114755 non-null  float64
 6   AMT_CREDIT                114755 non-null  float64
 7   AMT_ANNUITY               114755 non-null  float64
 8   NAME_INCOME_TYPE          114755 non-null  object 
 9   NAME_EDUCATION_TYPE       114755 non-null  object 
 10  NAME_FAMILY_STATUS        114755 non-null  object 
 11  NAME_HOUSING_TYPE         114755 non-null  object 
 12  DAYS_EMPLOYED             114755 non-null  int64  
 13  OCCUPATION_TYPE           114755 non-null  o

In [46]:
#replace value 0 in status with 2
# df["STATUS"].replace(0,2,inplace=True)
# df["STATUS"].replace(1,0,inplace=True)
# df["STATUS"].replace(2,1,inplace=True)
df["STATUS"].value_counts()


0    104221
1     10534
Name: STATUS, dtype: int64

In [47]:
df.dropna(inplace=True)
df.isnull().sum()

Unnamed: 0                  0
STATUS                      0
NAME_CONTRACT_TYPE          0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
AMT_INCOME_TOTAL            0
AMT_CREDIT                  0
AMT_ANNUITY                 0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_EMPLOYED               0
OCCUPATION_TYPE             0
CNT_FAM_MEMBERS             0
EXT_SOURCE_2                0
DEF_60_CNT_SOCIAL_CIRCLE    0
age                         0
total_enquiries_cb          0
credit_income_ratio         0
dtype: int64

In [48]:
data_dict = {}
for col in df.columns:
    data_dict[col] = [str(df[col].dtypes)]
    

In [49]:
object_list = []
for key, value in data_dict.items():
    if value == ['object']:
        object_list.append(key)
object_list.append('STATUS')
object_list

#label encode all object columns
for col in object_list:
    df[col] = LabelEncoder().fit_transform(df[col])

In [50]:
df.head()

,Unnamed: 0,STATUS,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,EXT_SOURCE_2,DEF_60_CNT_SOCIAL_CIRCLE,age,total_enquiries_cb,credit_income_ratio
0,0,0,0,1,1,171000.0,1560726.0,41301.0,1,1,1,1,-3130,0,3.0,0.724000,0.0,37.0,4.0,9.127053
1,2,0,0,0,1,112500.0,652500.0,21177.0,5,1,1,1,-679,3,3.0,0.651862,0.0,27.0,1.0,5.800000
2,3,0,0,0,1,67500.0,80865.0,5881.5,5,4,1,1,-2717,8,2.0,0.715042,0.0,36.0,1.0,1.198000
3,4,0,0,1,0,225000.0,918468.0,28966.5,5,4,1,1,-3028,4,3.0,0.566907,0.0,38.0,1.0,4.082080
4,6,0,0,0,0,108000.0,509602.5,26149.5,5,4,1,1,-1317,4,2.0,0.236378,0.0,35.0,1.0,4.718542


In [51]:
del df['Unnamed: 0']

In [52]:
# automatically calculate bin ranges
# bins = sc.woebin(df, y='STATUS',positive="bad|0")
bins = sc.woebin(df, y='STATUS')
# make it easy to read the bins
for variables , bindetails in bins.items():
    print(variables , " : ")
    display(bindetails)
    print("--"*50)

[INFO] creating woe binning ...
Binning on 114564 rows and 19 columns in 00:00:15
credit_income_ratio  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,credit_income_ratio,"[-inf,1.0)",6166,0.053821,5763,403,0.065358,-0.367799,0.006269,0.017477,1.0,False
1,credit_income_ratio,"[1.0,1.6)",13397,0.116939,12328,1069,0.079794,-0.152671,0.002561,0.017477,1.6,False
2,credit_income_ratio,"[1.6,2.2)",15601,0.136177,14188,1413,0.090571,-0.014203,0.000027,0.017477,2.2,False
3,credit_income_ratio,"[2.2,6.2)",62738,0.547624,56445,6293,0.100306,0.098649,0.005548,0.017477,6.2,False
4,credit_income_ratio,"[6.2,inf)",16662,0.145438,15329,1333,0.080002,-0.149836,0.003072,0.017477,inf,False


----------------------------------------------------------------------------------------------------
AMT_ANNUITY  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,AMT_ANNUITY,"[-inf,14000.0)",19409,0.169416,17979,1430,0.073677,-0.239052,8.782527e-03,0.032036,14000.0,False
1,AMT_ANNUITY,"[14000.0,36000.0)",67860,0.592333,60889,6971,0.102726,0.125184,9.769004e-03,0.032036,36000.0,False
2,AMT_ANNUITY,"[36000.0,42000.0)",10646,0.092926,9668,978,0.091865,0.001411,1.851115e-07,0.032036,42000.0,False
3,AMT_ANNUITY,"[42000.0,inf)",16649,0.145325,15517,1132,0.067992,-0.325472,1.348406e-02,0.032036,inf,False


----------------------------------------------------------------------------------------------------
NAME_INCOME_TYPE  : 


...

----------------------------------------------------------------------------------------------------
AMT_CREDIT  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,AMT_CREDIT,"[-inf,150000.0)",6177,0.053917,5756,421,0.068156,-0.322887,0.004929,0.067077,150000.0,False
1,AMT_CREDIT,"[150000.0,300000.0)",26222,0.228885,23953,2269,0.086530,-0.064276,0.000921,0.067077,300000.0,False
2,AMT_CREDIT,"[300000.0,650000.0)",39699,0.346522,35029,4670,0.117635,0.277461,0.029869,0.067077,650000.0,False
3,AMT_CREDIT,"[650000.0,850000.0)",16348,0.142698,14927,1421,0.086922,-0.059333,0.000490,0.067077,850000.0,False
4,AMT_CREDIT,"[850000.0,1350000.0)",20145,0.175841,18698,1447,0.071829,-0.266446,0.011199,0.067077,1350000.0,False
5,AMT_CREDIT,"[1350000.0,inf)",5973,0.052137,5690,283,0.047380,-0.708541,0.019669,0.067077,inf,False


----------------------------------------------------------------------------------------------------
NAME_FAMILY_STATUS  : 


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NAME_FAMILY_STATUS,"[-inf,1.0)",12185,0.106360,10877,1308,0.107345,0.174327,0.003470,0.009239,1.0,False
1,NAME_FAMILY_STATUS,"[1.0,2.0)",78550,0.685643,71755,6795,0.086505,-0.064592,0.002786,0.009239,2.0,False
2,NAME_FAMILY_STATUS,"[2.0,3.0)",6580,0.057435,5961,619,0.094073,0.027590,0.000044,0.009239,3.0,False
3,NAME_FAMILY_STATUS,"[3.0,inf)",17249,0.150562,15460,1789,0.103716,0.135879,0.002938,0.009239,inf,False


----------------------------------------------------------------------------------------------------


In [53]:
#drop credit_income_ratio
df.drop("credit_income_ratio",axis=1,inplace=True)
df.drop("total_enquiries_cb",axis=1,inplace=True)

In [54]:
df.drop("FLAG_OWN_REALTY",axis=1,inplace=True)
df.drop("NAME_HOUSING_TYPE",axis=1,inplace=True)

In [55]:
print(df.shape)
df.head()

(114564, 15)


,STATUS,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,EXT_SOURCE_2,DEF_60_CNT_SOCIAL_CIRCLE,age
0,0,0,1,171000.0,1560726.0,41301.0,1,1,1,-3130,0,3.0,0.724000,0.0,37.0
1,0,0,0,112500.0,652500.0,21177.0,5,1,1,-679,3,3.0,0.651862,0.0,27.0
2,0,0,0,67500.0,80865.0,5881.5,5,4,1,-2717,8,2.0,0.715042,0.0,36.0
3,0,0,1,225000.0,918468.0,28966.5,5,4,1,-3028,4,3.0,0.566907,0.0,38.0
4,0,0,0,108000.0,509602.5,26149.5,5,4,1,-1317,4,2.0,0.236378,0.0,35.0


In [56]:
breaks_adj = {
       'OCCUPATION_TYPE' : [1, 4, 15],
       'NAME_INCOME_TYPE' : [0,1,5],
        'NAME_EDUCATION_TYPE' : [0,2,3],
        'AMT_INCOME_TOTAL' : [200000,250000, 300000,400000],
        'DAYS_EMPLOYED' : [-5000,-3500, -2200, -1700, -1000],
        'AMT_ANNUITY' : [4000, 7000, 16000],
        'EXT_SOURCE_2' : [0.1,0.15,0.20,0.44,0.6,0.80],
        'AMT_CREDIT' : [91000,150000,200000,300000],
        'CNT_FAM_MEMBERS' : [1,2,3],
    'NAME_FAMILY_STATUS' : [0,3],
        'age' : [27, 30,35, 40],
        'DEF_60_CNT_SOCIAL_CIRCLE' : [0,1,2],
        'FLAG_OWN_CAR': [0,1]
        
    }
bins_final = sc.woebin(df, y='STATUS',breaks_list=breaks_adj)
# bins_final = sc.woebin(df, y='STATUS',breaks_list=breaks_adj, positive="bad|0")

[INFO] creating woe binning ...


In [57]:
# sample code
train, test = sc.split_df(df, 'STATUS', ratio=0.8).values()
print(train.shape)
print(test.shape)

(91651, 15)
(22913, 15)


In [58]:
print('Proportion of positive cases in train set:', train['STATUS'].mean())
print('Proportion of positive cases in test set:', test['STATUS'].mean())

Proportion of positive cases in train set: 0.09175022640233058
Proportion of positive cases in test set: 0.09173831449395539


In [59]:
# train_woe = sc.woebin_ply(train, bins_final, positive="bad|0")
# test_woe = sc.woebin_ply(test, bins_final, positive="bad|0")
train_woe = sc.woebin_ply(train, bins_final)
test_woe = sc.woebin_ply(test, bins_final)

[INFO] converting into woe values ...
[INFO] converting into woe values ...


In [60]:
y_train = train_woe.loc[:,'STATUS']
X_train = train_woe.loc[:,train_woe.columns != 'STATUS']
y_test = test_woe.loc[:,'STATUS']
X_test = test_woe.loc[:,test_woe.columns != 'STATUS']

In [62]:
print("Class distribution in original train set:", Counter(y_train))
rus = RandomUnderSampler(random_state=7)
X_train_rus_resampled, y_train_rus_resampled = rus.fit_resample(X_train, y_train)
print("Class distribution in resampled train set:", Counter(y_train_rus_resampled))

Class distribution in original train set: Counter({0: 83242, 1: 8409})
Class distribution in resampled train set: Counter({0: 8409, 1: 8409})


In [63]:
print("Class distribution in original train set:", Counter(y_train))
ros = RandomOverSampler(random_state=7)
X_train_ros_resampled, y_train_ros_resampled = ros.fit_resample(X_train, y_train)
print("Class distribution in resampled train set:", Counter(y_train_ros_resampled))

Class distribution in original train set: Counter({0: 83242, 1: 8409})
Class distribution in resampled train set: Counter({0: 83242, 1: 83242})


In [64]:
print("Class distribution in original train set:", Counter(y_train))
smote = SMOTE(random_state=7)
X_train_smote_resampled, y_train_smote_resampled = smote.fit_resample(X_train, y_train)
print("Class distribution in resampled train set:", Counter(y_train_smote_resampled))

Class distribution in original train set: Counter({0: 83242, 1: 8409})
Class distribution in resampled train set: Counter({0: 83242, 1: 83242})


In [65]:
#create a logistic regression model object
lr = LogisticRegression(class_weight='balanced', random_state=7)
lr.fit(X_train, y_train)
print(lr.coef_)
print(lr.intercept_)

[[0.70762312 0.40179317 0.10333775 0.6074612  0.86220769 0.88966486
  0.50440388 0.86035346 0.61778868 0.06325317 0.74167581 1.17754017
  0.77701069 0.86615051]]
[-0.00390821]


1 is good
0 is default

In [66]:
accuracy = lr.score(X_test, y_test)
accuracy

0.6485837734037446

In [67]:
# make predictions on the test set
y_pred = lr.predict(X_test)

In [68]:
# assume y_true and y_pred are the true and predicted labels, respectively
cm = confusion_matrix(y_test, y_pred, labels=[0,1])

# create a dataframe from the confusion matrix
df_cm = pd.DataFrame(cm, index=['Good', 'Default'], columns=['Predict Good', 'Predict Default'])


# print the dataframe
print(df_cm)

         Predict Good  Predict Default
Good            13560             7251
Default           801             1301


True positive (TP)
The model predicts a good customer, and the client did not default <br>
False positive (FP)
The model predicts a good customer, but the client defaulted<br>
True negative (TN) 
The model predicts a default, and the client defaulted<br>
False negative (FN)
The model predicts a default, but the client did not default

In [69]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[0,1]).ravel()
print(tp,fn,fp,tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) #Given that we predict positive, how often is it really positive
recall = tp / (tp + fn) #How good at predicting positive
f1 = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp) # How good at predicting negative?

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity) 
print("F1:", f1)

13560 7251 801 1301
Accuracy: 0.6485837734037446
Precision: 0.9442239398370587
Recall: 0.6515784921435779
Specificity: 0.6189343482397717
F1: 0.7710678949164108


# RUS

In [72]:
lr_rus = LogisticRegression(C=1, solver='liblinear', random_state=7)
lr_rus.fit(X_train_rus_resampled, y_train_rus_resampled)
print(lr_rus.coef_)
print(lr_rus.intercept_)

[[ 0.70589589  0.71169941 -0.01191753  0.59143766  0.91442382  0.89155576
   0.44595473  0.85331905  0.65696645 -0.29524461  0.7783618   0.92488134
   0.79405883  0.77128414]]
[-0.00352012]


In [73]:
y_rus_pred = lr_rus.predict(X_test)
cm = confusion_matrix(y_test, y_rus_pred, labels=[0,1])
df_cm = pd.DataFrame(cm, index=['Good', 'Default'], columns=['Predict Good', 'Predict Default'])
print(df_cm)

         Predict Good  Predict Default
Good            13560             7251
Default           796             1306


In [74]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
tp, fn, fp, tn = confusion_matrix(y_test, y_rus_pred, labels=[0,1]).ravel()
print(tp,fn,fp,tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) #Given that we predict positive, how often is it really positive
recall = tp / (tp + fn) #How good at predicting positive
f1 = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp) # How good at predicting negative?

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity) 
print("F1:", f1)

13560 7251 796 1306
Accuracy: 0.6488019901366037
Precision: 0.9445528002229033
Recall: 0.6515784921435779
Specificity: 0.621313035204567
F1: 0.7711775243836552


# ROS

In [78]:
lr_ros = LogisticRegression(C=1, solver='liblinear', random_state=7)
lr_ros.fit(X_train_ros_resampled, y_train_ros_resampled)
print(lr_ros.coef_)
print(lr_ros.intercept_)

[[0.70648937 0.48547155 0.08827639 0.6064858  0.82844671 0.89045982
  0.5315791  0.89046923 0.63633696 0.12027072 0.7327792  1.11875963
  0.76256482 0.90786405]]
[-0.00543027]


In [79]:
lr_ros_pred = lr_ros.predict(X_test)
cm = confusion_matrix(y_test, lr_ros_pred, labels=[0,1])
df_cm = pd.DataFrame(cm, index=['Good', 'Default'], columns=['Predict Good', 'Predict Default'])
print(df_cm)

         Predict Good  Predict Default
Good            13547             7264
Default           804             1298


In [80]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
tp, fn, fp, tn = confusion_matrix(y_test, lr_ros_pred, labels=[0,1]).ravel()
print(tp,fn,fp,tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) #Given that we predict positive, how often is it really positive
recall = tp / (tp + fn) #How good at predicting positive
f1 = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp) # How good at predicting negative?

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity) 
print("F1:", f1)

13547 7264 804 1298
Accuracy: 0.6478854798585956
Precision: 0.9439760295449794
Recall: 0.6509538224977176
Specificity: 0.6175071360608944
F1: 0.7705477504123771


# Smote

In [83]:
lr_smote = LogisticRegression(C=1, solver='liblinear', random_state=7)
lr_smote.fit(X_train_smote_resampled, y_train_smote_resampled)
print(lr_smote.coef_)
print(lr_smote.intercept_)

[[ 0.72667343  0.70091772  0.39426325  0.6422244   0.99157793  0.88773873
   0.64308847  0.95449769  0.78345818 -1.05379915  0.61250193  1.63995605
   0.83115168  0.63239972]]
[-0.01549624]


In [84]:
y_smote_pred = lr_smote.predict(X_test)
cm = confusion_matrix(y_test, y_smote_pred, labels=[0,1])
df_cm = pd.DataFrame(cm, index=['Good', 'Default'], columns=['Predict Good', 'Predict Default'])
print(df_cm)

         Predict Good  Predict Default
Good            13509             7302
Default           797             1305


In [85]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
tp, fn, fp, tn = confusion_matrix(y_test, y_smote_pred, labels=[0,1]).ravel()
print(tp,fn,fp,tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) #Given that we predict positive, how often is it really positive
recall = tp / (tp + fn) #How good at predicting positive
f1 = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp) # How good at predicting negative?

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity) 
print("F1:", f1)

13509 7302 797 1305
Accuracy: 0.6465325361148693
Precision: 0.9442891094645604
Recall: 0.6491278650713564
Specificity: 0.620837297811608
F1: 0.7693709599339351


In [93]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
def calculate(y_test,y_pred):
    tp, fn, fp, tn = confusion_matrix(y_test, y_pred, labels=[0,1]).ravel()
    return tn/(tn+fp)

# Original dataset tuning

In [94]:
numbers = []
for i in range(1, 200):
    numbers.append(i / 100)

best_c = 1
spec = 0.6189343482397717

In [95]:
for i in numbers:
    lr = LogisticRegression(random_state=7,C= i, solver= 'liblinear', class_weight='balanced')
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    res = calculate(y_test,y_pred)
    if res > spec:
        best_c=i
        spec=res

In [96]:
print(best_c)

1


# RUS Tuning

In [88]:
numbers = []
for i in range(1, 200):
    numbers.append(i / 100)

best_c = 1
spec = 0.621313035204567

In [89]:
for i in numbers:
    lr = LogisticRegression(random_state=7,C= i, solver= 'liblinear')
    lr.fit(X_train_rus_resampled, y_train_rus_resampled)
    y_pred = lr.predict(X_test)
    res = calculate(y_test,y_pred)
    if res > spec:
        best_c=i
        spec=res

In [92]:
print(best_c)

1.14


In [ ]:
lr_rus = LogisticRegression(C=1, solver='liblinear', random_state=7)
lr_rus.fit(X_train_rus_resampled, y_train_rus_resampled)
print(lr_rus.coef_)
print(lr_rus.intercept_)


In [ ]:
y_rus_pred = lr_rus.predict(X_test)
cm = confusion_matrix(y_test, y_rus_pred, labels=[0,1])
df_cm = pd.DataFrame(cm, index=['Good', 'Default'], columns=['Predict Good', 'Predict Default'])
print(df_cm)

In [ ]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
tp, fn, fp, tn = confusion_matrix(y_test, y_rus_pred, labels=[0,1]).ravel()
print(tp,fn,fp,tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) #Given that we predict positive, how often is it really positive
recall = tp / (tp + fn) #How good at predicting positive
f1 = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp) # How good at predicting negative?

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity) 
print("F1:", f1)

# ROS Tuning

In [97]:
numbers = []
for i in range(1, 200):
    numbers.append(i / 100)

best_c = 1
spec = 0.6175071360608944

In [98]:
for i in numbers:
    lr = LogisticRegression(random_state=7,C= i, solver= 'liblinear')
    lr.fit(X_train_ros_resampled, y_train_ros_resampled)
    y_pred = lr.predict(X_test)
    res = calculate(y_test,y_pred)
    if res > spec:
        best_c=i
        spec=res

In [100]:
print(best_c)

0.21


In [101]:
lr = LogisticRegression(random_state=7,C= 0.21, solver= 'liblinear')
lr.fit(X_train_ros_resampled, y_train_ros_resampled)

LogisticRegression(C=0.21, random_state=7, solver='liblinear')

In [102]:
lr_ros_pred = lr_ros.predict(X_test)
cm = confusion_matrix(y_test, lr_ros_pred, labels=[0,1])
df_cm = pd.DataFrame(cm, index=['Good', 'Default'], columns=['Predict Good', 'Predict Default'])
print(df_cm)

         Predict Good  Predict Default
Good            13547             7264
Default           804             1298


In [103]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
tp, fn, fp, tn = confusion_matrix(y_test, lr_ros_pred, labels=[0,1]).ravel()
print(tp,fn,fp,tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) #Given that we predict positive, how often is it really positive
recall = tp / (tp + fn) #How good at predicting positive
f1 = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp) # How good at predicting negative?

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity) 
print("F1:", f1)

13547 7264 804 1298
Accuracy: 0.6478854798585956
Precision: 0.9439760295449794
Recall: 0.6509538224977176
Specificity: 0.6175071360608944
F1: 0.7705477504123771


# SMOTE Tuning

In [106]:
numbers = []
for i in range(1, 200):
    numbers.append(i / 100)

best_c = 1
spec = 0.620837297811608

In [107]:
for i in numbers:
    lr_smote = LogisticRegression(C=1, solver='liblinear', random_state=7)
    lr_smote.fit(X_train_smote_resampled, y_train_smote_resampled)
    y_pred = lr_smote.predict(X_test)
    res = calculate(y_test,y_pred)
    if res > spec:
        best_c=i
        spec=res

In [108]:
print(best_c)

1


In [109]:
#create a logistic regression model object
lr_smote = LogisticRegression(C=1, solver='liblinear', random_state=7)
lr_smote.fit(X_train_smote_resampled, y_train_smote_resampled)
print(lr.coef_)
print(lr.intercept_)

[[0.70566543 0.49478577 0.08798268 0.60388034 0.80621461 0.89021128
  0.52970181 0.88939432 0.63200623 0.13085229 0.73092283 1.09493738
  0.7591641  0.90077856]]
[-0.00541043]


In [110]:
y_smote_pred = lr_smote.predict(X_test)
cm = confusion_matrix(y_test, y_smote_pred, labels=[0,1])
df_cm = pd.DataFrame(cm, index=['Good', 'Default'], columns=['Predict Good', 'Predict Default'])
print(df_cm)

         Predict Good  Predict Default
Good            13509             7302
Default           797             1305


In [111]:
#import accuracy score from sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
tp, fn, fp, tn = confusion_matrix(y_test, y_smote_pred, labels=[0,1]).ravel()
print(tp,fn,fp,tn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) #Given that we predict positive, how often is it really positive
recall = tp / (tp + fn) #How good at predicting positive
f1 = 2 * (precision * recall) / (precision + recall)
specificity = tn / (tn + fp) # How good at predicting negative?

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity) 
print("F1:", f1)

13509 7302 797 1305
Accuracy: 0.6465325361148693
Precision: 0.9442891094645604
Recall: 0.6491278650713564
Specificity: 0.620837297811608
F1: 0.7693709599339351


BEST SMOTE
<br> 
Accuracy: 0.6465325361148693 <br> 
Precision: 0.9442891094645604 <br> 
Recall: 0.6491278650713564 <br> 
Specificity: 0.620837297811608 <br> 
F1: 0.7693709599339351 <br> 

<br>
    
BEST ROS
<br> 
Accuracy: 0.6478854798585956 <br> 
Precision: 0.9439760295449794 <br> 
Recall: 0.6509538224977176<br> 
Specificity: 0.6175071360608944<br> 
F1: 0.7705477504123771<br> 

<br> 
    
BEST RUS
<br> 
Accuracy: 0.6465325361148693<br> 
Precision: 0.9442891094645604<br>
Recall: 0.6491278650713564<br> 
Specificity: 0.620837297811608<br> 
F1: 0.7693709599339351

ORIGINAL DATASET 
<br>
Accuracy: 0.6485837734037446<br>
Precision: 0.9442239398370587<br>
Recall: 0.6515784921435779<br>
Specificity: 0.6189343482397717<br>
F1: 0.7710678949164108<br>

Since all methods had not much difference, we will still use the original dataset that is not balanced.